In [1]:
# TASK QUESTIONS:
# 2.1.1: What is the accuracy of this method?
#   26.49%
# 2.1.2: How has the accuracy changed now?
#   19.00%
# 2.1.3: Where/What is the bug?
#   It was a int overflow, it went from a 8 bit to a 32 bit (distance varable). 
#   Thus creating a overflow. This was fixed by converting the input arrays to 32 bits (.astype(int)).
# 2.2.2: What is the accuracy with best k?
#   [a]

import numpy as np
from matplotlib import pyplot as plt

In [2]:
from keras.datasets import mnist

KeyboardInterrupt: 

In [ ]:
(Xtr, Ltr), (X_test, L_test)=mnist.load_data()

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
[5 0 4 ... 5 6 8]


In [ ]:
Xtr.shape

(60000, 28, 28)

In [ ]:
#Traing phase
num_sample=500
Tr_set=Xtr[:num_sample,:,:]

Ltr_set=Ltr[:num_sample]

# Bugged: Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2])

# FIXED VIA .astype(int)
Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2])
Tr_set=Tr_set.astype(int)

#Tr_set=Tr_set.reshape(num_sample,Tr_set.shape[1]*Tr_set.shape[2]).astype()
Tr_set.shape

(500, 784)

In [ ]:

def predict_1NN_L1(X):
    num_test=X.shape[0] # = 10000
    Lpred=np.zeros(num_test, dtype=Ltr_set.dtype)
    
    for i in range(num_test):
        distances = np.sum(np.abs(Tr_set-X[i,:]),axis=1)
        min_index= np.argmin(distances)
        Lpred[i]=Ltr_set[min_index]
        
    return Lpred

In [ ]:
def predict_1NN_L2(X):
    num_test=X.shape[0] # = 10000
    Lpred=np.zeros(num_test, dtype=Ltr_set.dtype)
    
    for i in range(num_test):
        distances = np.sqrt(np.sum((  np.abs(Tr_set-X[i,:])**2  ),axis=1) )
        
        min_index= np.argmin(distances)
        Lpred[i]=Ltr_set[min_index]
    return Lpred

In [ ]:
def extended_knn(k, X):
    num_test=X.shape[0] # = 10000
    Lpred=np.zeros(num_test, dtype=Ltr_set.dtype)
    for i in range(num_test):
        distances = np.sum(np.abs(Tr_set-X[i,:]),axis=1)
        min_index= np.argmin(distances)
        Lpred[i]=Ltr_set[min_index]
    return Lpred

In [ ]:
def cv_3fold(X):
    pass # return best k

#best_k = cv_3fold()
#extended_knn(best_k, X)

In [ ]:
Test_images=X_test.reshape(X_test.shape[0],X_test.shape[1]* X_test.shape[2])
Labels_predicted=predict_1NN_L1(Test_images)

print("Accuracy:", np.mean(Labels_predicted==L_test))

(10000, 784)
(500, 784)


Accuracy: 0.811
